In [ ]:
!pip install git+https://github.com/tky823/ssspy.git

In [ ]:
import numpy as np
import scipy.signal as ss
import matplotlib.pyplot as plt
import IPython.display as ipd

In [ ]:
from ssspy.utils.dataset import download_sample_speech_data

In [ ]:
n_sources = 2
max_samples = 10 * 16000
sisec2010_tag = "dev1_female3"
n_fft, hop_length = 4096, 2048

In [ ]:
waveform_src_img = download_sample_speech_data(n_sources=n_sources, sisec2010_tag=sisec2010_tag, max_samples=max_samples, conv=True) # (n_channels, n_sources, n_samples)
waveform_mix = np.sum(waveform_src_img, axis=1)  # (n_channels, n_samples)

In [ ]:
for idx, waveform in enumerate(waveform_mix):
    print("Mixture: {}".format(idx + 1))
    display(ipd.Audio(waveform, rate=16000))
    print()

In [ ]:
from ssspy.transform import whiten
from ssspy.algorithm import projection_back
from ssspy.bss.fdica import AuxFDICA

In [ ]:
def contrast_fn(y):
    return 2 * np.abs(y)

def d_contrast_fn(y):
    return 2 * np.ones_like(y)

In [ ]:
fdica_ip2 = AuxFDICA(
    algorithm_spatial="IP2",
    contrast_fn=contrast_fn,
    d_contrast_fn=d_contrast_fn,
    should_apply_projection_back=False
)
print(fdica_ip2)

In [ ]:
_, _, spectrogram_mix = ss.stft(waveform_mix, window="hann", nperseg=n_fft, noverlap=n_fft-hop_length)

In [ ]:
spectrogram_mix_whitened = whiten(spectrogram_mix)
spectrogram_est = fdica_ip2(spectrogram_mix_whitened, n_iter=20)
spectrogram_est = projection_back(spectrogram_est, reference=spectrogram_mix)

In [ ]:
_, waveform_est = ss.istft(spectrogram_est, window="hann", nperseg=n_fft, noverlap=n_fft-hop_length)

In [ ]:
for idx, waveform in enumerate(waveform_est):
    print("Estimated source: {}".format(idx + 1))
    display(ipd.Audio(waveform, rate=16000))
    print()

In [ ]:
plt.figure()
plt.plot(fdica_ip2.loss)
plt.show()
plt.close()